## Exercício: Modelo de Linguagem (Bengio 2003) - MLP + Embeddings

Neste exercício iremos treinar uma rede neural similar a do Bengio 2003 para prever a próxima palavra de um texto, data as palavras anteriores como entrada. Esta tarefa é chamada de "Modelagem da Linguagem".

Portanto, você deve implementar o modelo de linguagem inspirado no artigo do Bengio, para prever a próxima palavra usando rede com embeddings e duas camadas.
Sugestão de alguns parâmetros:
* context_size = 9
* max_vocab_size = 3000
* embedding_dim = 64
* usar pontuação no vocabulário
* descartar qualquer contexto ou target que não esteja no vocabulário
* É esperado conseguir uma perplexidade da ordem de 50.
* Procurem fazer asserts para garantir que partes do seu programa estão testadas

Este enunciado não é fixo, podem mudar qualquer um dos parâmetros acima, mas procurem conseguir a perplexidade esperada ou menor.

Gerem alguns frases usando um contexto inicial e depois deslocando o contexto e prevendo a próxima palavra gerando frases compridas para ver se está gerando texto plausível.

Algumas dicas:
- Inclua caracteres de pontuação (ex: `.` e `,`) no vocabulário.
- Deixe tudo como caixa baixa (lower-case).
- A escolha do tamanho do vocabulario é importante: ser for muito grande, fica difícil para o modelo aprender boas representações. Se for muito pequeno, o modelo apenas conseguirá gerar textos simples.
- Remova qualquer exemplo de treino/validação/teste que tenha pelo menos um token desconhecido (ou na entrada ou na saída).
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

Procure por `TODO` para entender onde você precisa inserir o seu código.

## Faz download e carrega o dataset

In [ ]:
import os

if not os.path.exists("data"):
    os.mkdir("data")
    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8 -P data/
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8 -P data/

In [ ]:
text = (
    open("data/67724.txt.utf-8", "r").read() + open("data/67725.txt.utf-8", "r").read()
)

paragraphs = text.split("\n\n")
len(paragraphs)

In [ ]:
cleaned_paragraphs = [
    paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()
]
len(cleaned_paragraphs)

## Análise do dataset

In [ ]:
# Conta as palavras no dataset
from collections import Counter
import re

regular_expression = re.compile(r"\w+|[.,!?-]")


def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(regular_expression, text.lower()))
    return word_counts


word_counts = count_words(cleaned_paragraphs)

print(len(word_counts))
print(word_counts)

## Criando um vocabulário

In [ ]:
vocab_size = 3000
most_frequent_words = ["<unk>"] + [
    word for word, count in word_counts.most_common(vocab_size)
]
vocab = {word: i for i, word in enumerate(most_frequent_words)}
vocab_size += 1

In [ ]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(r"\w+|[.,!?-]", sentence.lower())]


def decode_sentence(sentence, most_frequent_words):
    return " ".join([most_frequent_words[c] for c in sentence])


text = cleaned_paragraphs[0]

code = encode_sentence(text, vocab)
decode = decode_sentence(code, most_frequent_words)

print(code)
print(decode)

## Classe do dataset

In [ ]:
context_size = 5  # 5 palavras de entrada. O target é a próxima palavra
max_vocab_size = 3000
embedding_dim = 64
debug = 1064

In [ ]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

torch.manual_seed(18)


class MyDataset(Dataset):
    def __init__(self, data, vocab, context_size):
        self.data = data
        self.vocab = vocab
        self.context_size = context_size
        self.x = []
        self.y = []
        for sentence in data:
            words = encode_sentence(sentence, vocab)
            len_sample = len(words)
            if len_sample < context_size:
                continue
            for i in range(len(words) - context_size):
                if 0 in words[i : i + context_size] or 0 == words[i + context_size]:
                    continue
                self.x.append(words[i : i + context_size])
                self.y.append(words[i + context_size])

        self.x = torch.tensor(self.x)
        self.y = torch.tensor(self.y)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


# if debug limit data to 10 samples
if debug:
    train = train[:debug]
    val = val[:debug]

train_data = MyDataset(train, vocab, context_size)
val_data = MyDataset(val, vocab, context_size)

In [ ]:
batch_size = 128
train_loader = DataLoader(
    train_data, batch_size=batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, drop_last=True)

## Model

In [ ]:
import torch.nn as nn


class LanguageModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, hidden=128):
        super(LanguageModel, self).__init__()

        # Look up table
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Linear layer
        self.linear1 = nn.Linear(context_size * embedding_dim, hidden)
        self.relu = nn.ReLU()
        # Linear layer
        self.linear2 = nn.Linear(hidden, vocab_size)

    def forward(self, x):
        if len(x.shape) > 2:
            x = x.view(x.shape[0], -1)
        if len(x.shape) == 1:
            x = x.unsqueeze(0)
        x = self.embedding(x)
        x = x.view(x.shape[0], -1)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x

In [ ]:
model = LanguageModel(
    vocab_size, embedding_dim=128, context_size=context_size, hidden=256
)

In [ ]:
# number of parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

## Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

### Initial Loss and PPL

In [ ]:
epochs = 10
lr = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
model.to(device)

In [ ]:
model.eval()
total_loss = 0
with torch.no_grad():
    accuracy = 0
    for _input, target in val_loader:
        _input, target = _input.to(device), target.to(device)
        output = model(_input)
        loss = criterion(output, target)
        accuracy += (output.argmax(dim=1) == target).sum().item()
        total_loss += loss.item()

print(f"Initial Val Loss: {total_loss/len(val_loader)}")
print(f"Initial Val Accuracy: {accuracy/len(val_loader):.3f}%")
perplexity = torch.exp(torch.tensor(total_loss / len(val_loader)))
print(f"Initial Val Perplexity: {perplexity.item()}")

In [ ]:
for epoch in range(epochs):
    model.train()
    total_loss = 0
    accuracy = 0
    for i, (_input, target) in enumerate(train_loader):
        _input, target = _input.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(_input)
        loss = criterion(output, target)
        total_loss += loss.item()
        accuracy += (output.argmax(dim=1) == target).sum()/batch_size
        loss.backward()
        optimizer.step()
    
    # mean accuracy by the number of batches
    accuracy /= 
    perplexity = torch.exp(torch.tensor(total_loss/len(train_loader)))
    print(f"Epoch {epoch+1}/{epochs}\nTrain Loss: {total_loss/len(train_loader)}, Train Perplexity: {perplexity.item()}, Train Accuracy: {accuracy/len(train_loader):.3f}%")

    model.eval()
    total_loss = 0
    accuracy = 0
    with torch.no_grad():
        for i, (_input, target) in enumerate(val_loader):
            _input, target = _input.to(device), target.to(device)
            output = model(_input)
            loss = criterion(output, target)
            accuracy += (output.argmax(dim=1) == target).sum().item()
            total_loss += loss.item()

    perplexity = torch.exp(torch.tensor(total_loss/len(val_loader)))
    print(f"Validation Loss: {total_loss/len(val_loader)}, Validation Perplexity: {perplexity.item()}, Validation Accuracy: {accuracy/len(val_loader):.3f}%\n")

## Avaliação

In [ ]:
""" TODO: calcule a perplexidade final no dataset de validação """

# Perplexidade final

model.eval()
total_loss = 0
with torch.no_grad():
    for i, (_input, target) in enumerate(val_loader):
        _input, target = _input.to(device), target.to(device)
        output = model(_input)
        loss = criterion(output, target)
        total_loss += loss.item()

perplexity = torch.exp(torch.tensor(total_loss / len(val_loader)))
print(f"Final Perplexity: {perplexity.item()}")

## Exemplo de uso

In [ ]:
text = "um dia a praia irá"


def generate_text(model, vocab, text, max_length):
    """TODO: implemente a função para gerar texto até atingir o max_length"""
    model.eval()
    words = text.split(" ")
    for i in range(max_length):
        input_ids = encode_sentence(" ".join(text.split()[-context_size:]), vocab)
        _input = torch.tensor([input_ids]).to(device)
        output = model(_input)
        word = decode_sentence([output.argmax(dim=1).item()], most_frequent_words)
        words.append(word)
    return " ".join(words)


context = 5
max_length = 15
generate_text(model, vocab, text, max_length)

In [ ]:
val_loader